# Modelo Dimensional
#### Este notebook cria um csv para cada tabela de dimensão e um para a tabela de factos

In [65]:
import pandas as pd
import ast
from itertools import product
from textblob import TextBlob

In [66]:
calendar = pd.read_csv("cleaned_data/calendar.csv")
listings = pd.read_csv("cleaned_data/listings.csv")
parks = pd.read_csv("cleaned_data/parks.csv")
reviews = pd.read_csv("cleaned_data/reviews.csv")
weather = pd.read_csv("cleaned_data/weather.csv")

# Construção do modelo dimensional - tabelas dimAmenities

In [67]:
# Function to extract amenities from string
def extract_amenities(row):
    amenities = row['amenities']
    if amenities == '{}':
        return []
    else:
        return [amenity.strip('"') for amenity in amenities.strip('{}').split(',')]

# Extracting amenities and creating one-hot encoded DataFrame
amenities_list = listings.apply(extract_amenities, axis=1)
dimAmenitiesOneHot = pd.get_dummies(amenities_list.apply(pd.Series).stack()).sum(level=0)

# Create a row with zeros
new_row = pd.Series(0, index=dimAmenitiesOneHot.columns)

# Append the new row to dimAmenitiesOneHot
dimAmenitiesOneHot = dimAmenitiesOneHot.append(new_row, ignore_index=True)

# Create the dimAmenitiesOneHot index column
dimAmenitiesOneHot.insert(0, 'amenitiesPK', range(1, len(dimAmenitiesOneHot) + 1))

C:\Users\Rita\AppData\Local\Temp\ipykernel_24280\3950293420.py:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dimAmenitiesOneHot = pd.get_dummies(amenities_list.apply(pd.Series).stack()).sum(level=0)
C:\Users\Rita\AppData\Local\Temp\ipykernel_24280\3950293420.py:11: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dimAmenitiesOneHot = pd.get_dummies(amenities_list.apply(pd.Series).stack()).sum(level=0)
C:\Users\Rita\AppData\Local\Temp\ipykernel_24280\3950293420.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dimAmenitiesOneHot = dimAmenitiesOneHot.append(new_row, ignore_index=True)


In [68]:
dimAmenitiesOneHot

,amenitiesPK,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),Doorman,...,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
1,2,0,0,0,1,0,1,0,0,0,...,1,0,1,0,0,1,1,0,0,1
2,3,0,1,0,0,1,1,1,1,0,...,0,1,1,0,0,1,1,0,0,1
3,4,0,0,0,0,0,1,0,0,0,...,1,1,1,0,0,0,1,0,0,1
4,5,0,0,0,0,1,1,0,0,0,...,0,1,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3742,3743,1,0,0,0,1,0,0,0,0,...,0,1,1,0,0,1,0,0,0,1
3743,3744,0,0,1,0,1,0,1,1,0,...,1,0,1,0,0,0,1,0,0,1
3744,3745,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
3745,3746,0,0,0,1,1,0,0,1,0,...,0,1,1,0,0,1,1,0,0,1


# Construção do modelo dimensional - tabela dimDate

In [69]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...
...,...,...,...,...,...,...
84826,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...
84827,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...
84828,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del..."
84829,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si..."


In [70]:
# Create the "dimDate" DataFrame
dimDate = pd.DataFrame()

In [71]:
# Extract the date from the "date" column
dimDate['date'] = pd.to_datetime(reviews['date']).dt.date

# Extract year, month, and day from the "date" column
dimDate['year'] = pd.to_datetime(reviews['date']).dt.year
dimDate['monthNumber'] = pd.to_datetime(reviews['date']).dt.month
dimDate["month"] = pd.to_datetime(reviews['date']).dt.strftime('%B')
dimDate['day'] = pd.to_datetime(reviews['date']).dt.day

# Get unique rows
dimDate = dimDate.drop_duplicates()

# Extract the weather season based on the month
dimDate['weatherSeason'] = pd.to_datetime(reviews['date']).dt.strftime('%B').apply(
    lambda x: 'Spring' if x in ['March', 'April', 'May'] else
    'Summer' if x in ['June', 'July', 'August'] else
    'Autumn' if x in ['September', 'October', 'November'] else
    'Winter'
)

# Extract the weekday (monday = 1)
dimDate['weekDayNumber'] = pd.to_datetime(reviews['date']).dt.weekday + 1

dimDate['weekDay'] = pd.to_datetime(reviews['date']).dt.strftime('%A')

# Create weekend indicator ('weekday' for weekdays, 'weekend' for weekends)
dimDate['weekendIndicator'] = pd.to_datetime(reviews['date']).dt.dayofweek.map({0: 'weekday', 1: 'weekday', 2: 'weekday', 3: 'weekday', 4: 'weekday', 5: 'weekend', 6: 'weekend'})

# Determine if it is the tourism season (April to August and one week before and after Christmas)
dimDate['tourismSeason'] = dimDate.apply(lambda row: row['month'] in [4, 5, 6, 7, 8] or (row['month'] == 12 and 15 <= row['day'] <= 31), axis=1)

# Map the tourism season to 'high' and 'low' values
dimDate['tourismSeason'] = dimDate['tourismSeason'].map({True: 'high', False: 'low'})

# Sort the DataFrame in ascending order based on the "date" column
dimDate = dimDate.sort_values('date')

# Add an auto-incremental column "datePK"
dimDate.reset_index(drop=True, inplace=True)
dimDate.insert(0, 'datePK', dimDate.index + 1)


In [72]:
dimDate

,datePK,date,year,monthNumber,month,day,weatherSeason,weekDayNumber,weekDay,weekendIndicator,tourismSeason
0,1,2009-06-07,2009,6,June,7,Summer,7,Sunday,weekend,low
1,2,2009-06-28,2009,6,June,28,Summer,7,Sunday,weekend,low
2,3,2009-07-17,2009,7,July,17,Summer,5,Friday,weekday,low
3,4,2009-08-31,2009,8,August,31,Summer,1,Monday,weekday,low
4,5,2009-09-10,2009,9,September,10,Autumn,4,Thursday,weekday,low
...,...,...,...,...,...,...,...,...,...,...,...
1924,1925,2015-12-30,2015,12,December,30,Winter,3,Wednesday,weekday,low
1925,1926,2015-12-31,2015,12,December,31,Winter,4,Thursday,weekday,low
1926,1927,2016-01-01,2016,1,January,1,Winter,5,Friday,weekday,low
1927,1928,2016-01-02,2016,1,January,2,Winter,6,Saturday,weekend,low


In [73]:
# Save the "dimDate" DataFrame to a CSV file
dimDate.to_csv('dimensional_modeling/dimDate.csv', index=False)

# Construção do modelo dimensional - tabela dimProperty

In [74]:
listings.isna().sum().sum() #1
listings.dropna(inplace = True)
listings.isna().sum().sum() #0

0

In [75]:
# Select the desired columns from the "listings" DataFrame
dimProperty = listings[["id", "name", "number_of_reviews", "first_review", "last_review",
                        "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness",
                        "review_scores_checkin", "review_scores_communication", "review_scores_location",
                        "review_scores_value", "reviews_per_month"]]

# Rename the columns in the "dimProperty" DataFrame
dimProperty = dimProperty.rename(columns={"id": "propertyID", "name": "propertyName", "number_of_reviews": "numberOfReviews",
                                          "first_review": "firstReview", "last_review": "lastReview",
                                          "review_scores_rating": "reviewScoresRating",
                                          "review_scores_accuracy": "reviewScoresAccuracy",
                                          "review_scores_cleanliness": "reviewScoreCleanliness",
                                          "review_scores_checkin": "reviewScoresCheckIn",
                                          "review_scores_communication": "reviewScoresCommunication",
                                          "review_scores_location": "reviewScoresLocation",
                                          "review_scores_value": "reviewScoresValues",
                                          "reviews_per_month": "reviewsPerMonth"})

dimProperty['propertyName'] = dimProperty['propertyName'].str.replace("'", "")

# Drop duplicate rows based on all columns
dimProperty = dimProperty.drop_duplicates()

# Sort the DataFrame based on the "propertyID" column in ascending order
dimProperty = dimProperty.sort_values(by="propertyID")

# Add an auto-incremental column "propertyPK"
dimProperty.reset_index(drop=True, inplace=True)
dimProperty.insert(0, "propertyPK", range(1, len(dimProperty) + 1))

In [76]:
dimProperty

,propertyPK,propertyID,propertyName,numberOfReviews,firstReview,lastReview,reviewScoresRating,reviewScoresAccuracy,reviewScoreCleanliness,reviewScoresCheckIn,reviewScoresCommunication,reviewScoresLocation,reviewScoresValues,reviewsPerMonth
0,1,4291,Sunrise in Seattle Master Suite,35,2013-07-01,2015-10-18,92.000000,10.000000,9.000000,10.000000,9.000000,9.000000,9.000000,1.140000
1,2,5682,"Cozy Studio, min. to downtown -WiFi",297,2010-03-21,2015-12-14,96.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000,4.210000
2,3,6606,"Fab, private seattle urban cottage!",52,2009-07-17,2015-12-28,93.000000,9.000000,9.000000,10.000000,9.000000,10.000000,9.000000,0.660000
3,4,7369,launchingpad/landingpad,40,2009-06-07,2012-03-04,94.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,0.500000
4,5,9419,Golden Sun vintage warm/sunny,79,2010-07-30,2015-12-07,91.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,3782,10332096,Room & bath in suburban N Seattle,0,2014-08-13,2015-09-28,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.078919
3782,3783,10334184,Historic Capitol Hill Garden Apt.,0,2014-09-06,2015-09-07,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.078919
3783,3784,10339144,Studio in the heart of Capitol Hill,0,2012-08-05,2015-12-08,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.078919
3784,3785,10339145,West Seattle Beachfront Apartment,0,2015-10-11,2015-11-29,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.078919


In [77]:
# Export the DataFrame to a CSV file
dimProperty.to_csv("dimensional_modeling/dimProperty.csv", index=False)

# Construção do modelo dimensional - tabela dimHost

In [78]:
# Select the desired columns from the "listings" DataFrame
dimHost = listings[['host_id', 'host_since', 'host_response_time', 'host_response_rate', 'host_acceptance_rate',
                    'host_total_listings_count', 'host_verifications', 'host_identity_verified']]

# Rename the columns in the "dimHost" DataFrame
dimHost = dimHost.rename(columns={'host_id': 'hostID', 'host_since': 'hostSince', 'host_response_time': 'hostResponseTime',
                                  'host_response_rate': 'hostResponseRate', 'host_acceptance_rate': 'hostAcceptanceRate',
                                  'host_total_listings_count': 'hostTotalListings', 'host_verifications': 'hostVerifications',
                                  'host_identity_verified': 'hostIdentityVerified'})

# Drop duplicate rows based on all columns
dimHost = dimHost.drop_duplicates()

# Convert the "hostVerification" column to the number of elements in each list
dimHost['hostVerifications'] = dimHost['hostVerifications'].apply(lambda x: len(ast.literal_eval(x)))

# Sort the DataFrame based on the "hostID" column in ascending order
dimHost = dimHost.sort_values(by='hostID')

# Add an auto-incremental column "hostPK"
dimHost.reset_index(drop=True, inplace=True)
dimHost.insert(0, 'hostPK', range(1, len(dimHost) + 1))

In [79]:
dimHost

,hostPK,hostID,hostSince,hostResponseTime,hostResponseRate,hostAcceptanceRate,hostTotalListings,hostVerifications,hostIdentityVerified
0,1,4193,2008-11-10,within a few hours,0.880000,1.000000,4,5,Host Identity Verified
1,2,6207,2009-01-08,within an hour,1.000000,1.000000,1,6,Host Identity Verified
2,3,8021,2009-02-16,within a few hours,0.750000,1.000000,4,6,Host Identity Verified
3,4,8993,2009-03-03,within an hour,1.000000,1.000000,1,4,Host Identity Verified
4,5,11775,2009-03-30,within a few hours,1.000000,1.000000,1,5,Host Identity Verified
...,...,...,...,...,...,...,...,...,...
2735,2736,52990042,2016-01-01,within an hour,0.948868,0.999672,1,5,Host Identity Verified
2736,2737,53050379,2016-01-02,within an hour,0.948868,0.999672,1,3,Host Identity Not Verified
2737,2738,53065829,2016-01-02,within an hour,0.948868,0.999672,1,2,Host Identity Not Verified
2738,2739,53169216,2016-01-03,within an hour,0.948868,0.999672,1,2,Host Identity Not Verified


In [80]:
# Export the DataFrame to a CSV file
dimHost.to_csv('dimensional_modeling/dimHost.csv', index=False)

# Construção do modelo dimensional - tabela dimLocation

In [81]:
# Select the desired columns from the "listings" DataFrame
dimLocation = listings[['street', 'neighbourhood_group_cleansed', 'neighbourhood_cleansed', 'is_location_exact',
                        'zipcode', 'latitude', 'longitude']]

# Rename the columns in the "dimLocation" DataFrame
dimLocation = dimLocation.rename(columns={'street': 'street', 'neighbourhood_group_cleansed': 'neighbourhoodGroupCleansed',
                                          'neighbourhood_cleansed': 'neighbourhoodCleansed',
                                          'is_location_exact': 'isLocationExact', 'zipcode': 'zipcode',
                                          'latitude': 'latitude', 'longitude': 'longitude'})

# Drop duplicate rows based on all columns
dimLocation = dimLocation.drop_duplicates()

# Add an auto-incremental column "streetPK"
dimLocation.reset_index(drop=True, inplace=True)
dimLocation.insert(0, 'streetPK', range(1, len(dimLocation) + 1))

In [82]:
from math import sin, cos, sqrt, atan2, radians

# Define the radius in kilometers
radius = 1.0

# Function to calculate the distance between two coordinates using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates to radians
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Distance in kilometers
    distance_km = 6371 * c
    return distance_km

# Iterate over each location in dimLocation and count the number of nearby parks
dimLocation['nearbyParksCount'] = 0  # Initialize the nearbyParksCount column to zero
for index, row in dimLocation.iterrows():
    count = 0
    for _, park in parks.iterrows():
        distance = calculate_distance(
            row['latitude'],
            row['longitude'],
            park['Y Coord'],
            park['X Coord']
        )
        if distance <= radius:
            count += 1
    dimLocation.at[index, 'nearbyParksCount'] = count

In [83]:
dimLocation

,streetPK,street,neighbourhoodGroupCleansed,neighbourhoodCleansed,isLocationExact,zipcode,latitude,longitude,nearbyParksCount
0,1,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Location is Exact,98119,47.636289,-122.371025,8
1,2,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Location is Exact,98119,47.639123,-122.365666,8
2,3,"West Lee Street, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Location is Exact,98119,47.629724,-122.369483,9
3,4,"8th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Location is Exact,98119,47.638473,-122.369279,8
4,5,"14th Ave W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Location is Exact,98119,47.632918,-122.372471,9
...,...,...,...,...,...,...,...,...,...
3781,3782,"Northwest 48th Street, Seattle, WA 98107, Unit...",Other neighborhoods,Fremont,Location is Exact,98107,47.664295,-122.359170,9
3782,3783,"Fuhrman Avenue East, Seattle, WA 98102, United...",Capitol Hill,Portage Bay,Location is Exact,98102,47.649552,-122.318309,11
3783,3784,"South Laurel Street, Seattle, WA 98178, United...",Rainier Valley,Rainier Beach,Location is Approximate,98178,47.508453,-122.240607,3
3784,3785,"43rd Avenue East, Seattle, WA 98112, United St...",Capitol Hill,Madison Park,Location is Approximate,98112,47.632335,-122.275530,5


In [84]:
# Export the DataFrame to a CSV file
dimLocation.to_csv('dimensional_modeling/dimLocation.csv', index=False)

# Construção do modelo dimensional - tabelas dimFacilities

In [85]:
# Select the desired columns from the "listings" DataFrame
dimFacilities = listings[['property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
                          'guests_included', 'amenities']]

# Rename the columns in the "dimFacilities" DataFrame
dimFacilities = dimFacilities.rename(columns={'property_type': 'propertyType', 'room_type': 'roomType',
                                              'bed_type': 'bedType', 'guests_included': 'guestsIncluded'})

# Drop duplicate rows based on all columns
dimFacilities = dimFacilities.drop_duplicates()

# Sort the DataFrame based on the "propertyType" column in ascending order
dimFacilities = dimFacilities.sort_values(by='propertyType')

# Add an auto-incremental column "facilitiesPK"
dimFacilities.reset_index(drop=True, inplace=True)
dimFacilities.insert(0, 'facilitiesPK', range(1, len(dimFacilities) + 1))

In [86]:
dimFacilities

,facilitiesPK,propertyType,roomType,accommodates,bathrooms,bedrooms,beds,bedType,guestsIncluded,amenities
0,1,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A..."
1,2,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,1,"{""Wireless Internet"",Kitchen,Heating,Washer,Dr..."
2,3,Apartment,Entire home/apt,4,1.0,1.0,3.0,Real Bed,1,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A..."
3,4,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,1,"{TV,""Cable TV"",""Wireless Internet"",Kitchen,Hea..."
4,5,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,1,"{TV,Internet,""Wireless Internet"",""Air Conditio..."
...,...,...,...,...,...,...,...,...,...,...
3640,3641,Townhouse,Entire home/apt,6,3.5,3.0,5.0,Real Bed,6,"{TV,""Wireless Internet"",Kitchen,""Free Parking ..."
3641,3642,Treehouse,Entire home/apt,2,1.0,0.0,1.0,Real Bed,1,"{Internet,""Wireless Internet"",Kitchen,""Free Pa..."
3642,3643,Treehouse,Private room,1,1.0,1.0,1.0,Real Bed,1,{}
3643,3644,Treehouse,Private room,2,1.0,1.0,1.0,Real Bed,1,"{""Wireless Internet"",Kitchen,""Free Parking on ..."


In [87]:
# Function to map amenities dictionary to amenitiesFK
def map_amenities(row):
    amenities = row['amenities']
    if pd.notnull(amenities):
        amenities = amenities.replace('"', '')  # Remove double quotes if present
        amenities = amenities.replace('{', '')  # Remove opening brace if present
        amenities = amenities.replace('}', '')  # Remove closing brace if present
        amenities_list = amenities.split(',')
        amenities_list = [amenity.strip() for amenity in amenities_list]
        if amenities_list != [""]:
            non_amenities_cols = [col for col in dimAmenitiesOneHot.columns if col != 'amenitiesPK' and col not in amenities_list]
            match_rows = (dimAmenitiesOneHot[amenities_list] == 1).all(axis=1) & (dimAmenitiesOneHot[non_amenities_cols] == 0).all(axis=1)
            if match_rows.any():
                matching_row = dimAmenitiesOneHot.loc[match_rows]
                return int(matching_row['amenitiesPK'].values[0])
        else:
            return 3619
    return None

# Creating the amenitiesFK column in dimFacilities
dimFacilities['amenitiesFK'] = dimFacilities.apply(map_amenities, axis=1)

In [88]:
dimFacilities.dropna(inplace=True)
dimFacilities = dimFacilities.drop("amenities", axis=1)

In [89]:
dimFacilities

,facilitiesPK,propertyType,roomType,accommodates,bathrooms,bedrooms,beds,bedType,guestsIncluded,amenitiesFK
0,1,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,2,1
1,2,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,1,1670
2,3,Apartment,Entire home/apt,4,1.0,1.0,3.0,Real Bed,1,1669
3,4,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,1,1667
4,5,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,1,1666
...,...,...,...,...,...,...,...,...,...,...
3640,3641,Townhouse,Entire home/apt,6,3.5,3.0,5.0,Real Bed,6,1827
3641,3642,Treehouse,Entire home/apt,2,1.0,0.0,1.0,Real Bed,1,2518
3642,3643,Treehouse,Private room,1,1.0,1.0,1.0,Real Bed,1,3619
3643,3644,Treehouse,Private room,2,1.0,1.0,1.0,Real Bed,1,2146


In [90]:
# Export the DataFrame to a CSV file
dimFacilities.to_csv('dimensional_modeling/dimFacilities.csv', index=False)# Export the DataFrame to a CSV file

In [91]:
# Original column names
original_columns = list(dimAmenitiesOneHot.columns)

# Create a mapping of old column names to new column names
column_mapping = {col: col.lower().replace('/', '').replace('(s)', '').replace('-', '')
                  for col in original_columns[1:]}

# Rename the columns
dimAmenitiesOneHot.rename(columns=column_mapping, inplace=True)


# Update the column names with desired format
dimAmenitiesOneHot.columns = [col.split(' ')[0] + ''.join(word.capitalize() for word in col.split(' ')[1:])
                              for col in dimAmenitiesOneHot.columns]


dimAmenitiesOneHot.rename(columns={"24hourCheckin": "alldaycheckin"}, inplace=True)

In [92]:
# Export the DataFrame to a CSV file
dimAmenitiesOneHot.to_csv('dimensional_modeling/dimAmenities.csv', index=False)

# Construção do modelo dimensional - tabela dimPriceAndBooking

In [93]:
# Select the desired columns from the "listings" DataFrame
dimPriceAndBooking = listings[['instant_bookable', 'cancellation_policy']]

# Rename the columns in the "dimPriceAndBooking" DataFrame
dimPriceAndBooking = dimPriceAndBooking.rename(columns={'instant_bookable': 'instantBookable',
                                                        'cancellation_policy': 'cancellationPolicy'})

# Drop duplicate rows based on all columns
dimPriceAndBooking = dimPriceAndBooking.drop_duplicates()

# Define the different ranges
price_ranges = ['0-49.99', '50-99.99', '100-149.99', '150-199.99', '200-299.99', '>300']

# Generate all combinations of 'instantBookable', 'cancellationPolicy', and 'price_ranges'
combinations = list(product(dimPriceAndBooking['instantBookable'], dimPriceAndBooking['cancellationPolicy'], price_ranges))

# Create a new DataFrame for the combinations
dimPriceAndBookingCombinations = pd.DataFrame(combinations, columns=['instantBookable', 'cancellationPolicy', 'nightPriceRange'])

# Merge the new DataFrame with the original DataFrame
dimPriceAndBooking = dimPriceAndBooking.merge(dimPriceAndBookingCombinations, on=['instantBookable', 'cancellationPolicy'], how='outer')

# Drop duplicate rows based on all columns
dimPriceAndBooking = dimPriceAndBooking.drop_duplicates()

# Add an auto-incremental key column
dimPriceAndBooking.reset_index(drop=True, inplace=True)
dimPriceAndBooking.insert(0, 'nightPricePK', range(1, len(dimPriceAndBooking) + 1))


In [94]:
dimPriceAndBooking

,nightPricePK,instantBookable,cancellationPolicy,nightPriceRange
0,1,Instant Bookable Not Available,moderate,0-49.99
1,2,Instant Bookable Not Available,moderate,50-99.99
2,3,Instant Bookable Not Available,moderate,100-149.99
3,4,Instant Bookable Not Available,moderate,150-199.99
4,5,Instant Bookable Not Available,moderate,200-299.99
5,6,Instant Bookable Not Available,moderate,>300
6,7,Instant Bookable Not Available,strict,0-49.99
7,8,Instant Bookable Not Available,strict,50-99.99
8,9,Instant Bookable Not Available,strict,100-149.99
9,10,Instant Bookable Not Available,strict,150-199.99


In [95]:
# Export the DataFrame to a CSV file
dimPriceAndBooking.to_csv('dimensional_modeling/dimPriceAndBooking.csv', index=False)

# Construção do modelo dimensional - tabela dimWeather

In [96]:
weather

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,It rained
1,1948-01-02,0.59,45,36,It rained
2,1948-01-03,0.42,45,35,It rained
3,1948-01-04,0.31,45,34,It rained
4,1948-01-05,0.17,45,32,It rained
...,...,...,...,...,...
25543,2017-12-10,0.00,49,34,It did not rain
25544,2017-12-11,0.00,49,29,It did not rain
25545,2017-12-12,0.00,46,32,It did not rain
25546,2017-12-13,0.00,48,34,It did not rain


In [97]:
# Define the possible values for each column in Fahrenheit
tMinBand_values = ['>=14', '15-32', '33-50', '51-68', '69-86', '>=87']
tMaxBand_values = ['>=14', '15-32', '33-50', '51-68', '69-86', '>=87']
precipitationBand_values = ['0-0.99', '1-2.99', '3-4.99', '>=5']
rain_values = ['It rained', 'It did not rain']

# Generate all combinations of the values
combinations = list(product(tMinBand_values, tMaxBand_values, precipitationBand_values, rain_values))

# Create the "dimWeather" DataFrame
dimWeather = pd.DataFrame(combinations, columns=['tMinBand', 'tMaxBand', 'precipitationBand', 'rain'])

# Add an auto-incremental column "weatherPK"
dimWeather.insert(0, 'weatherPK', range(1, len(dimWeather) + 1))

In [98]:
dimWeather

,weatherPK,tMinBand,tMaxBand,precipitationBand,rain
0,1,>=14,>=14,0-0.99,It rained
1,2,>=14,>=14,0-0.99,It did not rain
2,3,>=14,>=14,1-2.99,It rained
3,4,>=14,>=14,1-2.99,It did not rain
4,5,>=14,>=14,3-4.99,It rained
...,...,...,...,...,...
283,284,>=87,>=87,1-2.99,It did not rain
284,285,>=87,>=87,3-4.99,It rained
285,286,>=87,>=87,3-4.99,It did not rain
286,287,>=87,>=87,>=5,It rained


In [99]:
# Export the DataFrame to a CSV file
dimWeather.to_csv('dimensional_modeling/dimWeather.csv', index=False)

# Construção do modelo dimensional - tabela dimCustomer

In [100]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...
...,...,...,...,...,...,...
84826,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...
84827,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...
84828,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del..."
84829,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si..."


In [101]:
# Select the desired columns from the "reviews" DataFrame
dimCustomer = reviews[['reviewer_id', 'reviewer_name']]

# Rename the columns in the "dimCustomer" DataFrame
dimCustomer = dimCustomer.rename(columns={'reviewer_id': 'reviewerID', 'reviewer_name': 'reviewerName'})

# Remove quotation marks
dimCustomer['reviewerName'] = dimCustomer['reviewerName'].str.replace("'", "")

# Drop duplicate rows based on all columns
dimCustomer = dimCustomer.drop_duplicates()

# Sort the DataFrame based on the "reviewerID" column in ascending order
dimCustomer = dimCustomer.sort_values(by='reviewerID')

# Add an auto-incremental column "reviewerPK"
dimCustomer.reset_index(drop=True, inplace=True)
dimCustomer.insert(0, 'reviewerPK', range(1, len(dimCustomer) + 1))

In [102]:
dimCustomer

,reviewerPK,reviewerID,reviewerName
0,1,15,Kat
1,2,262,Jonathan
2,3,431,Matthew
3,4,1618,Elaine
4,5,1720,Ryan
...,...,...,...
75712,75713,52713564,Mary
75713,75714,52721080,Ashley
75714,75715,52723379,Xavi
75715,75716,52790726,Paul


In [103]:
# Export the DataFrame to a CSV file
dimCustomer.to_csv('dimensional_modeling/dimCustomer.csv', index=False)

# Construção do modelo dimensional - fact table staysFact

In [141]:
reviews = pd.read_csv("cleaned_data/reviews.csv")
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...
...,...,...,...,...,...,...
84826,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...
84827,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...
84828,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del..."
84829,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si..."


In [142]:
# Perform sentiment analysis on comments
reviews['stayEvaluation'] = reviews['comments'].apply(lambda x: TextBlob(x).sentiment.polarity)

reviews['date'] = pd.to_datetime(reviews['date'])
dimDate["date"] = pd.to_datetime(dimDate["date"])

# Perform the merge and add the "dateFK" column to "reviews"
reviews = pd.merge(reviews, dimDate[['date', 'datePK']], on='date', how='left')
reviews = reviews.rename(columns={'datePK': 'dateFK'})

# Merge "reviews" with "dimProperty" on the "listing_id" and "propertyID" columns
reviews = pd.merge(reviews, dimProperty[['propertyID', 'propertyPK']], left_on='listing_id', right_on='propertyID', how='left')

# Rename the "propertyPK" column to "propertyFK"
reviews = reviews.rename(columns={'propertyPK': 'propertyFK'})

reviews = pd.merge(reviews, listings[['id', 'host_id']], left_on='listing_id', right_on='id', how='left')
reviews = reviews.rename(columns={'id_x': 'id'})
reviews = reviews.rename(columns={'id_y': 'listings_id'})

reviews = pd.merge(reviews, dimHost[['hostPK', 'hostID']], left_on='host_id', right_on='hostID', how='left')
reviews = reviews.rename(columns={'hostPK': 'hostFK'})

# Merge "reviews" with "listings" on the "listing_id" column
reviews = pd.merge(reviews, listings[['id', 'latitude', 'longitude']], left_on='listing_id', right_on='id', how='left')
reviews = reviews.rename(columns={'id_x': 'id'})
reviews = reviews.rename(columns={'id_y': 'listings_id'})

# Merge the intermediate result with "dimLocation" based on the "latitude" and "longitude" columns
reviews = pd.merge(reviews, dimLocation[['latitude', 'longitude', 'streetPK']], on=['latitude', 'longitude'], how='left')

# Rename the "streetPK" column to "streetFK"
reviews = reviews.rename(columns={'streetPK': 'streetFK'})

reviews.drop(["comments", "reviewer_name", "propertyID", "host_id", "latitude", "longitude", "listings_id"], axis = 1, inplace = True)

reviews = pd.merge(reviews, dimCustomer[['reviewerID', 'reviewerPK']], left_on='reviewer_id', right_on='reviewerID', how='left')

reviews = reviews.rename(columns={'reviewerPK': 'customerFK'})

In [143]:
listings['facilitiesPK'] = None

# Iterate over each row in the "listings" dataframe
for index, row in listings.iterrows():
    # Extract the relevant values from the current row
    property_type = row['property_type']
    room_type = row['room_type']
    accommodates = row['accommodates']
    bathrooms = row['bathrooms']
    bedrooms = row['bedrooms']
    beds = row['beds']
    bed_type = row['bed_type']
    guests_included = row['guests_included']
    amenities = row['amenities']
    
    # Find a matching facility in the "dimFacilities" dataframe based on the values
    matching_facility = dimFacilities[
        (dimFacilities['propertyType'] == property_type) &
        (dimFacilities['roomType'] == room_type) &
        (dimFacilities['accommodates'] == accommodates) &
        (dimFacilities['bathrooms'] == bathrooms) &
        (dimFacilities['bedrooms'] == bedrooms) &
        (dimFacilities['beds'] == beds) &
        (dimFacilities['bedType'] == bed_type) &
        (dimFacilities['guestsIncluded'] == guests_included)
    ]
    

    # Get the corresponding facility primary key value
    facility_pk = matching_facility.iloc[0]['facilitiesPK']
    
    # Assign the facility primary key value to the "facilitiesPK" column in the "listings" dataframe
    listings.at[index, 'facilitiesPK'] = facility_pk

reviews = pd.merge(reviews, listings[['id', 'facilitiesPK']], left_on='listing_id', right_on='id', how='left')
reviews = reviews.rename(columns={'id_x': 'id'})
reviews = reviews.rename(columns={'id_y': 'listings_id'})
reviews = reviews.rename(columns={'facilitiesPK': 'facilitiesFK'})

In [144]:
# Create an empty "nightPricePK" column in the "listings" dataframe
listings['nightPricePK'] = None

# Iterate over each row in the "listings" dataframe
for index, row in listings.iterrows():
    # Extract the values from the current row
    instant_bookable = row['instant_bookable']
    cancellation_policy = row['cancellation_policy']
    price = row['price']
    
    # Find a matching row in the "dimPriceAndBooking" dataframe
    matching_row = dimPriceAndBooking[
        (dimPriceAndBooking['instantBookable'] == instant_bookable) &
        (dimPriceAndBooking['cancellationPolicy'] == cancellation_policy)
    ]
    
    # Check if a matching row is found
    if not matching_row.empty:
        # Iterate over the "nightPriceRange" values in the matching row
        for range_value in matching_row['nightPriceRange']:
            # Handle the special case of ">300" range value
            if range_value == '>300':

                if price > 300:

                    matching_rows = matching_row[matching_row['nightPriceRange'] == range_value]

                    # Get the corresponding value of nightPricePK
                    night_price_pk = matching_rows.iloc[0]['nightPricePK']
                    
                    # Assign the nightPricePK value to the "nightPricePK" column in the "listings" dataframe
                    listings.at[index, 'nightPricePK'] = night_price_pk
                    break
            else:
                # Extract the lower and upper bounds of the range

                range_bounds = range_value.split('-')
                lower_bound = int(float(range_bounds[0]))
                upper_bound = float(range_bounds[1])
                
                # Check if the price falls within the range
                if lower_bound <= price <= upper_bound:

                    range_str = "-".join([str(lower_bound), str(upper_bound)])

                    matching_rows = matching_row[matching_row['nightPriceRange'] == range_str]

                    # Get the corresponding value of nightPricePK
                    night_price_pk = matching_rows.iloc[0]['nightPricePK']
                    
                    # Assign the nightPricePK value to the "nightPricePK" column in the "listings" dataframe
                    listings.at[index, 'nightPricePK'] = night_price_pk
                    break  # Exit the loop once a matching range is found

In [145]:
reviews = pd.merge(reviews, listings[['id', 'nightPricePK']], left_on='listing_id', right_on='id', how='left')
reviews = reviews.rename(columns={'id_x': 'id'})
reviews = reviews.rename(columns={'id_y': 'listings_id'})
reviews = reviews.rename(columns={'nightPricePK': 'nightPriceFK'})

In [146]:
# Convert the "DATE" column to datetime format
weather['DATE'] = pd.to_datetime(weather['DATE'])

# Filter the dataset to keep only dates starting from 2009-06-07
weather = weather[weather['DATE'] >= '2009-06-07']

# Reset the index of the filtered DataFrame
weather.reset_index(drop=True, inplace=True)

In [147]:
# Define the temperature and precipitation bands
tMinBand_values = ['>=14', '15-32', '33-50', '51-68', '69-86', '>=87']
tMaxBand_values = ['>=14', '15-32', '33-50', '51-68', '69-86', '>=87']
precipitationBand_values = ['0-0.99', '1-2.99', '3-4.99', '>=5']

# Define a function to assign the weatherPK based on temperature bands
def assign_weather_pk(row):
    t_min = row['TMIN']
    t_max = row['TMAX']
    precipitation = row['PRCP']
    rain = row['RAIN']
    
    # Find the matching weatherPK based on temperature bands
    if t_min >= 87:
        t_min_band = '>=87'
    elif t_min >= 68:
        t_min_band = '69-86'
    elif t_min >= 50:
        t_min_band = '51-68'
    elif t_min >= 32:
        t_min_band = '33-50'
    elif t_min >= -49:
        t_min_band = '15-32'
    else:
        t_min_band = '<-49'
    
    if t_max >= 87:
        t_max_band = '>=87'
    elif t_max >= 68:
        t_max_band = '69-86'
    elif t_max >= 50:
        t_max_band = '51-68'
    elif t_max >= 32:
        t_max_band = '33-50'
    elif t_max >= -49:
        t_max_band = '15-32'
    else:
        t_max_band = '<-49'
    
    # Find the matching weatherPK based on precipitation band

    if precipitation >= 5:
        precipitation_band = ">=5"
    elif precipitation >= 3:
        precipitation_band = "3-4.99"
    elif precipitation >= 1:
        precipitation_band = "1-2.99"
    else:
        precipitation_band = "0-0.99"
  
    # Find the matching row in dimWeather based on temperature bands and precipitation band
    matching_row = dimWeather[
        (dimWeather['tMinBand'] == t_min_band) &
        (dimWeather['tMaxBand'] == t_max_band) &
        (dimWeather['precipitationBand'] == precipitation_band) &
        (dimWeather['rain'] == rain)
    ]
    
    # Return the weatherPK value if a matching row is found
    if not matching_row.empty:
        return matching_row.iloc[0]['weatherPK']
    else:
        return None

# Apply the assign_weather_pk function to create the "weatherPK" column
weather['weatherPK'] = weather.apply(assign_weather_pk, axis=1)

reviews = pd.merge(reviews, weather[['DATE', 'weatherPK']], left_on='date', right_on="DATE", how='left')
reviews = reviews.rename(columns={'weatherPK': 'weatherFK'})

In [148]:
reviews['count'] = 1

In [149]:
reviews.drop(["listing_id", "id", "date", "reviewer_id", "hostID", "reviewerID", "listings_id", "DATE"], axis = 1, inplace = True)

# Define the desired column order
desired_columns = ['dateFK', 'propertyFK', 'nightPriceFK', 'customerFK', 'streetFK', 'hostFK', 'facilitiesFK', 'weatherFK', 'stayEvaluation', 'count']

# Reorder the columns in the DataFrame
reviews = reviews.reindex(columns=desired_columns)

# Add the "stayFK" column as the first column
reviews.insert(0, "stayPK", range(1, len(reviews) + 1))

In [150]:
reviews.dropna(inplace=True)
reviews["propertyFK"] = reviews["propertyFK"].astype(int)
reviews["nightPriceFK"] = reviews["nightPriceFK"].astype(int)
reviews["streetFK"] = reviews["streetFK"].astype(int)
reviews["hostFK"] = reviews["hostFK"].astype(int)
reviews["facilitiesFK"] = reviews["facilitiesFK"].astype(int)

In [151]:
reviews

,stayPK,dateFK,propertyFK,nightPriceFK,customerFK,streetFK,hostFK,facilitiesFK,weatherFK,stayEvaluation,count
0,1,1761,2373,32,57573,3196,2346,591,186,0.433333,1
1,2,1762,2373,32,62071,3196,2346,591,178,0.301136,1
2,3,1768,2373,32,68032,3196,2346,591,177,0.410417,1
3,4,1775,2373,32,63614,3196,2346,591,186,0.358333,1
4,5,1783,2373,32,65184,3196,2346,591,178,0.493485,1
...,...,...,...,...,...,...,...,...,...,...,...
84826,84827,1846,1050,28,67769,1567,1451,91,169,0.194844,1
84827,84828,1851,1050,28,23123,1567,1451,91,169,0.311111,1
84828,84829,1854,1050,28,45300,1567,1451,91,170,0.000000,1
84829,84830,1867,1050,28,51889,1567,1451,91,121,0.216852,1


In [152]:
# Export the DataFrame to a CSV file
reviews.to_csv('dimensional_modeling/staysFact.csv', index=False)